In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install datasets
!pip3 install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
import pandas as pd
from datasets import Dataset
from huggingface_hub import hf_hub_download
import os
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import shutil

In [ ]:

start_index = 0
end_index = 55000
batch_size = 55000
max_workers = 20

drive_path = "/content/drive/MyDrive/AMMeBa_downloads"
os.makedirs(drive_path, exist_ok=True)
save_dir = os.path.join(drive_path, f"images_{start_index}_{end_index}")
os.makedirs(save_dir, exist_ok=True)

In [ ]:

def download_csv_file(repo_id: str, filename: str, repo_type: str = "dataset") -> str:
    file_path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        repo_type=repo_type
    )
    return file_path


def load_clean_dataset(csv_path: str) -> Dataset:

    df = pd.read_csv(csv_path, na_values=["", " ", "  ", "NA", "nan", "NaN"])

    return Dataset.from_pandas(df)

dataset_name = "academic-datasets/AMMeBa"
data_files = ["image_metadata.csv", "fact_checks.csv", "stage_2.csv"]

clean_datasets = {}

for file in data_files:
    print(f"\n📥 Downloading {file} from {dataset_name} ...")
    path = download_csv_file(dataset_name, file)
    ds = load_clean_dataset(path)
    clean_datasets[file] = ds
    print(f"✅ Loaded {file}: {len(ds)} records, columns: {ds.column_names}")


📥 Downloading image_metadata.csv from academic-datasets/AMMeBa ...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


image_metadata.csv:   0%|          | 0.00/11.5M [00:00<?, ?B/s]

✅ Loaded image_metadata.csv: 50413 records, columns: ['image_id', 'mime', 'height', 'width', 'image_size_bytes', 'num_faces', 'total_face_area', 'has_text', 'url', 'average_hash', 'phash', 'colorhash']

📥 Downloading fact_checks.csv from academic-datasets/AMMeBa ...


fact_checks.csv:   0%|          | 0.00/29.2M [00:00<?, ?B/s]

✅ Loaded fact_checks.csv: 145594 records, columns: ['fact_check_url', 'review_date', 'fact_check_title']

📥 Downloading stage_2.csv from academic-datasets/AMMeBa ...


stage_2.csv:   0%|          | 0.00/33.3M [00:00<?, ?B/s]

✅ Loaded stage_2.csv: 86197 records, columns: ['Unnamed: 0', 'fact_check_url', 'misinfo_source', 'image_id', 'submission_time', 'disqualified', 'disqualification_reason', 'image_type', 'screenshot_type', 'image_watermark', 'image_annotation', 'image_text', 'image_text_misinfo_relevance', 'content_manipulation', 'text_manipulation', 'chyron_manipulation', 'ai_manipulation', 'context_manipulation', 'premanipulation_image', 'premanipulation_image_id', 'self_miscontextualizing', 'circumstance_manipulation', 'identity_manipulation', 'location_manipulation', 'datetime_manipulation', 'atypical_manipulation', 'reverse_image_search', 'image_of_screen', 'fake_official_document']


In [ ]:
clean_datasets["stage_2.csv"][0]

{'Unnamed: 0': 0,
 'fact_check_url': 'https://factcheck.afp.com/these-photos-show-baby-rescued-indonesian-boat-disaster-2018',
 'misinfo_source': 'https://perma.cc/6AZ7-QB7P?view-mode=client-side',
 'image_id': '990af75e218d0b2d277a971385ea833c729a110f1be50698bdce637e',
 'submission_time': '2023.02.23 12:59:21',
 'disqualified': False,
 'disqualification_reason': None,
 'image_type': 'complex',
 'screenshot_type': None,
 'image_watermark': None,
 'image_annotation': None,
 'image_text': 'legible',
 'image_text_misinfo_relevance': 'relevant',
 'content_manipulation': 'false',
 'text_manipulation': None,
 'chyron_manipulation': None,
 'ai_manipulation': None,
 'context_manipulation': 'true',
 'premanipulation_image': None,
 'premanipulation_image_id': None,
 'self_miscontextualizing': 'true',
 'circumstance_manipulation': 'true',
 'identity_manipulation': 'false',
 'location_manipulation': 'false',
 'datetime_manipulation': 'false',
 'atypical_manipulation': False,
 'reverse_image_search

In [ ]:
# Missing values from image_metadata
import numpy as np
print(clean_datasets["image_metadata.csv"].to_pandas().isna().sum())

image_id                0
mime                    0
height                280
width                 280
image_size_bytes        0
num_faces              82
total_face_area        82
has_text                0
url                 14807
average_hash          291
phash                 291
colorhash             291
dtype: int64


In [ ]:
df_stage2 = clean_datasets["stage_2.csv"].to_pandas()
df_metadata = clean_datasets["image_metadata.csv"].to_pandas()

# Merge image_id, Get the URL for download
df_full = pd.merge(df_stage2, df_metadata, on="image_id", how="inner")

mod_cols = [
    "content_manipulation",
]
df_filtered = df_full[df_full[mod_cols].any(axis=1)].reset_index(drop=True)
df_batch = df_filtered.iloc[start_index:end_index].copy()

print(f"📦 From {start_index} to {end_index}，total {len(df_batch)")


📦 本批任务：0 到 55000，共 53687 条


In [ ]:
# Convert string values to proper booleans
df_full["content_manipulation"] = df_full["content_manipulation"].astype(str).str.lower().map({"true": True, "false": False})

# Now count
num_content_manipulated = df_full["content_manipulation"].sum()
print(f"🧠 Number of images with content_manipulation = True: {num_content_manipulated}")


🧠 Number of images with content_manipulation = True: 12643


In [ ]:
# ------------ Main loop: Process one batch at a time ------------
gdrive_output_dir = "/content/drive/MyDrive/AMMeBa_batches"
os.makedirs(gdrive_output_dir, exist_ok=True)

def download_and_save_batch(batch_df, batch_start, batch_end):
    # Local directory setup
    batch_folder = f"/content/images_{batch_start}_{batch_end}"
    os.makedirs(batch_folder, exist_ok=True)

    # Image downloading function
    def download_image(row):
        img_url = row["url"]
        image_id = row["image_id"]
        filename = f"{image_id}.jpg"
        filepath = os.path.join(batch_folder, filename)

        try:
            response = requests.get(img_url, timeout=10)
            if response.status_code == 200:
                with open(filepath, "wb") as f:
                    f.write(response.content)
                return True, image_id
            else:
                return False, image_id
        except:
            return False, image_id

    print(f"Starting download for image batch {batch_start} - {batch_end} ...")
    total_tasks = len(batch_df)
    report_interval = max(int(total_tasks * 0.05), 1)
    completed = 0
    success_count = 0
    fail_count = 0
    successful_ids = []

    rows = batch_df.to_dict(orient="records")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(download_image, row): row["image_id"]
            for row in rows
        }

        for i, future in enumerate(tqdm(as_completed(futures), total=total_tasks)):
            success, image_id = future.result()
            completed += 1
            if success:
                success_count += 1
                successful_ids.append(image_id)
            else:
                fail_count += 1

            if completed % report_interval == 0 or completed == total_tasks:
                success_rate = success_count / completed * 100
                print(f"{completed}/{total_tasks} completed — Success: {success_count}, Fail: {fail_count}, Success rate: {success_rate:.2f}%")

    # Save labels
    df_downloaded = batch_df[batch_df["image_id"].isin(successful_ids)].copy()
    df_downloaded["filename"] = df_downloaded["image_id"].apply(lambda x: f"{x}.jpg")
    local_csv_path = f"/content/labels_{batch_start}_{batch_end}.csv"
    df_downloaded.to_csv(local_csv_path, index=False)

    # Compress images
    zip_path = f"/content/images_{batch_start}_{batch_end}.zip"
    shutil.make_archive(zip_path.replace(".zip", ""), 'zip', batch_folder)
    print(f"Compression complete: {zip_path}")

    # Upload to Google Drive
    shutil.copy(zip_path, gdrive_output_dir)
    shutil.copy(local_csv_path, gdrive_output_dir)
    print(f"Uploaded to Google Drive: {gdrive_output_dir}")

    # Cleanup temporary files
    shutil.rmtree(batch_folder)
    os.remove(zip_path)
    os.remove(local_csv_path)
    print(f"Cleanup completed for batch {batch_start}-{batch_end}")

# Execute multiple batches
for batch_start in range(start_index, end_index, batch_size):
    batch_end = min(batch_start + batch_size, end_index)
    batch_df = df_filtered.iloc[batch_start:batch_end].copy()

    if batch_df.empty:
        print(f"Skipping empty batch {batch_start}-{batch_end}")
        continue

    download_and_save_batch(batch_df, batch_start, batch_end)



🚀 开始下载图像 Batch 0 - 5000 ...


  5%|▌         | 263/5000 [00:04<01:17, 60.87it/s]

📊 250/5000 完成 — 成功：133，失败：117，成功率：53.20%


 10%|█         | 513/5000 [00:12<01:23, 53.58it/s]

📊 500/5000 完成 — 成功：279，失败：221，成功率：55.80%


...

100%|█████████▉| 3674/3687 [01:27<00:00, 43.49it/s]

📊 3680/3687 完成 — 成功：2193，失败：1487，成功率：59.59%


100%|██████████| 3687/3687 [01:37<00:00, 37.99it/s]

📊 3687/3687 完成 — 成功：2193，失败：1494，成功率：59.48%


✅ 压缩完成：/content/images_50000_55000.zip
📤 已上传到 Google Drive: /content/drive/MyDrive/AMMeBa_batches
🧹 清理完成 Batch 50000-55000


In [ ]:
import pandas as pd
import zipfile
import os

# Set paths
batch_dir = "/content/drive/MyDrive/AMMeBa_batches"
final_csv_path = os.path.join(batch_dir, "all_labels.csv")
final_zip_path = os.path.join(batch_dir, "all_images_batches.zip")

# Merge all CSV files
csv_files = sorted([f for f in os.listdir(batch_dir) if f.startswith("labels_") and f.endswith(".csv")])
df_all = pd.DataFrame()

for csv_file in csv_files:
    csv_path = os.path.join(batch_dir, csv_file)
    df = pd.read_csv(csv_path)
    df_all = pd.concat([df_all, df], ignore_index=True)

# Save the merged CSV file
df_all.to_csv(final_csv_path, index=False)
print(f"Merge complete. CSV saved to: {final_csv_path}, total records: {len(df_all)}")

# Combine all ZIP files (image archives) into one ZIP
zip_files = sorted([f for f in os.listdir(batch_dir) if f.startswith("images_") and f.endswith(".zip")])

with zipfile.ZipFile(final_zip_path, "w") as final_zip:
    for zip_file in zip_files:
        zip_path = os.path.join(batch_dir, zip_file)
        arcname = os.path.basename(zip_path)
        final_zip.write(zip_path, arcname)

print(f"All image zip files have been bundled into: {final_zip_path}")


✅ 合并完成，CSV 路径：/content/drive/MyDrive/AMMeBa_batches/all_labels.csv，共 30105 条记录
✅ 所有 ZIP 文件已打包为：/content/drive/MyDrive/AMMeBa_batches/all_images_batches.zip
